# Building the dataset of research papers

The [Entrez](http://biopython.org/DIST/docs/api/Bio.Entrez-module.html) module, a part of the [Biopython](http://biopython.org/) library, will be used to interface with [PubMed](http://www.ncbi.nlm.nih.gov/pubmed).<br>
You can download Biopython from [here](http://biopython.org/wiki/Download).

In this notebook we will be covering several of the steps taken in the [Biopython Tutorial](http://biopython.org/DIST/docs/tutorial/Tutorial.html), specifically in [Chapter 9  Accessing NCBI’s Entrez databases](http://biopython.org/DIST/docs/tutorial/Tutorial.html#htoc109).

In [1]:
from Bio import Entrez

# NCBI requires you to set your email address to make use of NCBI's E-utilities
Entrez.email = "Your.Name.Here@example.org"

The datasets will be saved as serialized Python objects, compressed with bzip2.
Saving/loading them will therefore require the [pickle](http://docs.python.org/3/library/pickle.html) and [bz2](http://docs.python.org/3/library/bz2.html) modules.

In [2]:
import pickle, bz2, os

## EInfo: Obtaining information about the Entrez databases

In [3]:
# accessing extended information about the PubMed database
pubmed = Entrez.read( Entrez.einfo(db="pubmed"), validate=False )[u'DbInfo']

# list of possible search fields for use with ESearch:
search_fields = { f['Name']:f['Description'] for f in pubmed["FieldList"] }

In search_fields, we find 'TIAB' ('Free text associated with Abstract/Title') as a possible search field to use in searches.

In [4]:
search_fields

{'ALL': 'All terms from all searchable fields',
 'UID': 'Unique number assigned to publication',
 'FILT': 'Limits the records',
 'TITL': 'Words in title of publication',
 'WORD': 'Free text associated with publication',
 'MESH': 'Medical Subject Headings assigned to publication',
 'MAJR': 'MeSH terms of major importance to publication',
 'AUTH': 'Author(s) of publication',
 'JOUR': 'Journal abbreviation of publication',
 'AFFL': "Author's institutional affiliation and address",
 'ECNO': 'EC number for enzyme or CAS registry number',
 'SUBS': 'CAS chemical name or MEDLINE Substance Name',
 'PDAT': 'Date of publication',
 'EDAT': 'Date publication first accessible through Entrez',
 'VOL': 'Volume number of publication',
 'PAGE': 'Page number(s) of publication',
 'PTYP': 'Type of publication (e.g., review)',
 'LANG': 'Language of publication',
 'ISS': 'Issue number of publication',
 'SUBH': 'Additional specificity for MeSH term',
 'SI': 'Cross-reference from publication to other databases

## ESearch: Searching the Entrez databases

To have a look at the kind of data we get when searching the database, we'll perform a search for papers authored by Haasdijk:

In [5]:
example_authors = ['Haasdijk E']
example_search = Entrez.read( Entrez.esearch( db="pubmed", term=' AND '.join([a+'[AUTH]' for a in example_authors]) ) )
example_search

{'Count': '36', 'RetMax': '20', 'RetStart': '0', 'IdList': ['29311830', '28513205', '28513201', '28323435', '28140628', '26933487', '24977986', '24901702', '24852945', '24708899', '24252306', '23580075', '23144668', '22174697', '22154920', '21870131', '21760539', '20662596', '20602234', '20386726'], 'TranslationSet': [], 'TranslationStack': [{'Term': 'Haasdijk E[Author]', 'Field': 'Author', 'Count': '36', 'Explode': 'N'}, 'GROUP'], 'QueryTranslation': 'Haasdijk E[Author]'}

Note how the result being produced is not in Python's native string format:

In [6]:
type( example_search['IdList'][0] )

Bio.Entrez.Parser.StringElement

The part of the query's result we are most interested in is accessible through

In [7]:
example_ids = [ int(id) for id in example_search['IdList'] ]
print(example_ids)

[29311830, 28513205, 28513201, 28323435, 28140628, 26933487, 24977986, 24901702, 24852945, 24708899, 24252306, 23580075, 23144668, 22174697, 22154920, 21870131, 21760539, 20662596, 20602234, 20386726]


### PubMed IDs dataset

We will now assemble a dataset comprised of research articles containing the given keyword, in either their titles or abstracts.

In [8]:
search_term = 'fungus'

In [9]:
Ids_file = 'data/' + search_term + '_Ids.pkl.bz2'

In [10]:
if os.path.exists( Ids_file ):
    Ids = pickle.load( bz2.BZ2File( Ids_file, 'rb' ) )
else:
    # determine the number of hits for the search term
    search = Entrez.read( Entrez.esearch( db="pubmed", term=search_term+'[TIAB]', retmax=0 ) )
    total = int( search['Count'] )
    
    # `Ids` will be incrementally assembled, by performing multiple queries,
    # each returning at most `retrieve_per_query` entries.
    Ids_str = []
    retrieve_per_query = 10000
    
    for start in range( 0, total, retrieve_per_query ):
        print('Fetching IDs of results [%d,%d]' % ( start, start+retrieve_per_query ) )
        s = Entrez.read( Entrez.esearch( db="pubmed", term=search_term+'[TIAB]', retstart=start, retmax=retrieve_per_query ) )
        Ids_str.extend( s[ u'IdList' ] )
    
    # convert Ids to integers (and ensure that the conversion is reversible)
    Ids = [ int(id) for id in Ids_str ]
    
    for (id_str, id_int) in zip(Ids_str, Ids):
        if str(id_int) != id_str:
            raise Exception('Conversion of PubMed ID %s from string to integer it not reversible.' % id_str )
    
    # Remove IDs that would cause problems below:
    Ids.remove(29352803)
    
    # Save list of Ids
    pickle.dump( Ids, bz2.BZ2File( Ids_file, 'wb' ) )
    
total = len( Ids )
print('%d documents contain the search term "%s".' % ( total, search_term ) )

54751 documents contain the search term "fungus".


Taking a look at what we just retrieved, here are the last 5 elements of the `Ids` list:

In [11]:
Ids[:5]

[33017218, 33016130, 33016123, 33016118, 33014692]

## ESummary: Retrieving summaries from primary IDs

To have a look at the kind of metadata we get from a call to `Entrez.esummary()`, we now fetch the summary of one of Haasdijk's papers (using one of the PubMed IDs we obtained in the previous section:

In [12]:
example_paper = Entrez.read( Entrez.esummary(db="pubmed", id='33017218') )[0]

def print_dict( p ):
    for k,v in p.items():
        print(k)
        print('\t', v)

print_dict(example_paper)

Item
	 []
Id
	 33017218
PubDate
	 2020 Dec
EPubDate
	 
Source
	 Virulence
AuthorList
	 ['Ding JL', 'Hou J', 'Feng MG', 'Ying SH']
LastAuthor
	 Ying SH
Title
	 Transcriptomic analyses reveal comprehensive responses of insect hemocytes to mycopathogen Beauveria bassiana, and fungal virulence-related cell wall protein assists pathogen to evade host cellular defense.
Volume
	 11
Issue
	 1
Pages
	 1352-1365
LangList
	 ['English']
NlmUniqueID
	 101531386
ISSN
	 2150-5594
ESSN
	 2150-5608
PubTypeList
	 ['Journal Article']
RecordStatus
	 PubMed - in process
PubStatus
	 ppublish
ArticleIds
	 {'pubmed': ['33017218'], 'medline': [], 'doi': '10.1080/21505594.2020.1827886', 'rid': '33017218', 'eid': '33017218'}
DOI
	 10.1080/21505594.2020.1827886
History
	 {'pubmed': ['2020/10/06 06:00'], 'medline': ['2020/10/06 06:00'], 'entrez': '2020/10/05 17:08'}
References
	 []
HasAbstract
	 IntegerElement(1, attributes={})
PmcRefCount
	 IntegerElement(0, attributes={})
FullJournalName
	 Virulence
ELocationID


For now, we'll keep just some basic information for each paper: title, list of authors, publication year, and [DOI](https://en.wikipedia.org/wiki/Digital_object_identifier).

In case you are not familiar with the DOI system, know that the paper above can be accessed through the link  `https://www.doi.org/` followed by the paper's DOI.

In [13]:
( example_paper['Title'], example_paper['AuthorList'], int(example_paper['PubDate'][:4]), example_paper['DOI'] )

('Transcriptomic analyses reveal comprehensive responses of insect hemocytes to mycopathogen Beauveria bassiana, and fungal virulence-related cell wall protein assists pathogen to evade host cellular defense.',
 ['Ding JL', 'Hou J', 'Feng MG', 'Ying SH'],
 2020,
 '10.1080/21505594.2020.1827886')

### Summaries dataset

We are now ready to assemble a dataset containing the summaries of all the paper `Ids` we previously fetched.

To reduce the memory footprint, and to ensure the saved datasets won't depend on Biopython being installed to be properly loaded, values returned by `Entrez.read()` will be converted to their corresponding native Python types. We start by defining a function for helping with the conversion of strings:

In [14]:
Summaries_file = 'data/' + search_term + '_Summaries.pkl.bz2'

In [15]:
if os.path.exists( Summaries_file ):
    Summaries = pickle.load( bz2.BZ2File( Summaries_file, 'rb' ) )
else:
    # `Summaries` will be incrementally assembled, by performing multiple queries,
    # each returning at most `retrieve_per_query` entries.
    Summaries = []
    retrieve_per_query = 500
    
    print('Fetching Summaries of results: ')
    for start in range( 0, len(Ids), retrieve_per_query ):
        if (start % 10000 == 0):
            print('')
            print(start, end='')
        else:
            print('.', end='')
        
        # build comma separated string with the ids at indexes [start, start+retrieve_per_query)
        query_ids = ','.join( [ str(id) for id in Ids[ start : start+retrieve_per_query ] ] )
        
        s = Entrez.read( Entrez.esummary( db="pubmed", id=query_ids ) )
        
        # out of the retrieved data, we will keep only a tuple (title, authors, year, DOI), associated with the paper's id.
        # (all values converted to native Python formats)
        for p in s:
            try:
                f = [
                    ( int( p['Id'] ), (
                        str( p['Title'] ),
                        [ str(a) for a in p['AuthorList'] ],
                        int( p['PubDate'][:4] ),                # keeps just the publication year
                        str( p.get('DOI', '') )            # papers for which no DOI is available get an empty string in their place
                        ) )
                    ]
                Summaries.extend( f )
            except ValueError as e:
                print("\nError with ID " + p['Id'] + ": " + str(e))
                print("Manually remove this ID above and re-run code.")
    
    # Save Summaries, as a dictionary indexed by Ids
    Summaries = dict( Summaries )
    
    pickle.dump( Summaries, bz2.BZ2File( Summaries_file, 'wb' ) )

Let us take a look at the first 3 retrieved summaries:

In [16]:
{ id : Summaries[id] for id in Ids[:3] }

{33017218: ('Transcriptomic analyses reveal comprehensive responses of insect hemocytes to mycopathogen Beauveria bassiana, and fungal virulence-related cell wall protein assists pathogen to evade host cellular defense.',
  ['Ding JL', 'Hou J', 'Feng MG', 'Ying SH'],
  2020,
  '10.1080/21505594.2020.1827886'),
 33016130: ('Cytotoxic xanthones from the plant endophytic fungus <i>Paecilamyces</i> sp. TE-540.',
  ['Li XQ', 'Dong QJ', 'Xu K', 'Yuan XL', 'Liu XM', 'Zhang P'],
  2020,
  '10.1080/14786419.2020.1828410'),
 33016123: ('One new piperazinedione isolated from a mangrove-derived fungus <i>Aspergillus niger</i> JX-5.',
  ['Bai M',
   'Wang Y',
   'Liu T',
   'Lian YX',
   'Bai QQ',
   'Song XP',
   'Han CR',
   'Zheng CJ',
   'Chen GY'],
  2020,
  '10.1080/14786419.2020.1828407')}

## EFetch: Downloading full records from Entrez

`Entrez.efetch()` is the function that will allow us to obtain paper abstracts. Let us start by taking a look at the kind of data it returns when we query PubMed's database.

In [17]:
q = Entrez.read( Entrez.efetch(db="pubmed", id='33017218', retmode="xml") )['PubmedArticle']

`q` is a list, with each member corresponding to a queried id. Because here we only queried for one id, its results are then in `q[0]`.

In [18]:
type(q), len(q)

(list, 1)

1
At `q[0]` we find a dictionary containing two keys, the contents of which we print below.

In [19]:
type(q[0]), q[0].keys()

(Bio.Entrez.Parser.DictionaryElement,
 dict_keys(['MedlineCitation', 'PubmedData']))

In [20]:
print_dict( q[0][ 'PubmedData' ] )

ReferenceList
	 []
History
	 [DictElement({'Year': '2020', 'Month': '10', 'Day': '5', 'Hour': '17', 'Minute': '8'}, attributes={'PubStatus': 'entrez'}), DictElement({'Year': '2020', 'Month': '10', 'Day': '6', 'Hour': '6', 'Minute': '0'}, attributes={'PubStatus': 'pubmed'}), DictElement({'Year': '2020', 'Month': '10', 'Day': '6', 'Hour': '6', 'Minute': '0'}, attributes={'PubStatus': 'medline'})]
PublicationStatus
	 ppublish
ArticleIdList
	 [StringElement('33017218', attributes={'IdType': 'pubmed'}), StringElement('10.1080/21505594.2020.1827886', attributes={'IdType': 'doi'})]


The key `'MedlineCitation'` maps into another dictionary. In that dictionary, most of the information is contained under the key `'Article'`. To minimize the clutter, below we show the contents of `'MedlineCitation'` excluding its `'Article'` member, and below that we then show the contents of `'Article'`.

In [21]:
print_dict( { k:v for k,v in q[0][ 'MedlineCitation' ].items() if k!='Article' } )

OtherID
	 []
SpaceFlightMission
	 []
CitationSubset
	 ['IM']
KeywordList
	 [ListElement([StringElement(' Beauveria bassiana ', attributes={'MajorTopicYN': 'N'}), StringElement('cell-wall protein', attributes={'MajorTopicYN': 'N'}), StringElement('encapsulation', attributes={'MajorTopicYN': 'N'}), StringElement('entomopathogenic fungus', attributes={'MajorTopicYN': 'N'}), StringElement('hemocyte response', attributes={'MajorTopicYN': 'N'}), StringElement('insect immunity', attributes={'MajorTopicYN': 'N'}), StringElement('virulence', attributes={'MajorTopicYN': 'N'})], attributes={'Owner': 'NOTNLM'})]
OtherAbstract
	 []
GeneralNote
	 []
PMID
	 33017218
DateRevised
	 {'Year': '2020', 'Month': '10', 'Day': '05'}
MedlineJournalInfo
	 {'Country': 'United States', 'MedlineTA': 'Virulence', 'NlmUniqueID': '101531386', 'ISSNLinking': '2150-5594'}


In [22]:
print_dict( q[0][ 'MedlineCitation' ][ 'Article' ] )

ELocationID
	 [StringElement('10.1080/21505594.2020.1827886', attributes={'EIdType': 'doi', 'ValidYN': 'Y'})]
Language
	 ['eng']
ArticleDate
	 []
Journal
	 {'ISSN': StringElement('2150-5608', attributes={'IssnType': 'Electronic'}), 'JournalIssue': DictElement({'Volume': '11', 'Issue': '1', 'PubDate': {'Year': '2020', 'Month': 'Dec'}}, attributes={'CitedMedium': 'Internet'}), 'Title': 'Virulence', 'ISOAbbreviation': 'Virulence'}
ArticleTitle
	 Transcriptomic analyses reveal comprehensive responses of insect hemocytes to mycopathogen Beauveria bassiana, and fungal virulence-related cell wall protein assists pathogen to evade host cellular defense.
Pagination
	 {'MedlinePgn': '1352-1365'}
Abstract
	 {'AbstractText': ['Entomopathogenic fungi naturally infect insect hosts in environment. Fungal invasion and host immune defense are still in the progress of co-evolution. In this study, entomopathogenic fungus <i>Beauveria bassiana</i> and lepidopteran insect <i>Galleria mellonella</i> were us

A paper's abstract can therefore be accessed with:

In [23]:
{ int(q[0]['MedlineCitation']['PMID']) : str(q[0]['MedlineCitation']['Article']['Abstract']['AbstractText'][0]) }

{33017218: 'Entomopathogenic fungi naturally infect insect hosts in environment. Fungal invasion and host immune defense are still in the progress of co-evolution. In this study, entomopathogenic fungus <i>Beauveria bassiana</i> and lepidopteran insect <i>Galleria mellonella</i> were used to investigate host cellular immunity and fungal strategy to evade host defense. First of all, genome-wide expression revealed the transcriptomic responses of hemocytes to insect mycopathogen, which dynamically varied during infection process. Enrichment analysis indicated that differentially expressed genes were primarily involved in metabolism, cellular process and immune system. Notably, cellular response involved a series of hydrolytic enzyme and antimicrobial peptide genes which were sorted together in clustering analysis. In <i>B. bassiana</i>, a cell-wall protein gene (<i>BbCwp</i>) contributes to fungal development in host hemocoel and virulence. RT-qPCR analyses indicated that infection by Δ<

Some of the ids in our dataset refer to books from the [NCBI Bookshelf](http://www.ncbi.nlm.nih.gov/books/), a collection of freely available, downloadable, on-line versions of selected biomedical books. For such ids, `Entrez.efetch()` returns a slightly different structure, where the keys `[u'BookDocument', u'PubmedBookData']` take the place of the `[u'MedlineCitation', u'PubmedData']` keys we saw above.

### Abstracts dataset

We can now assemble a dataset mapping paper ids to their abstracts.

In [24]:
Abstracts_file = 'data/' + search_term + '_Abstracts.pkl.bz2'

In [25]:
import http.client
from collections import deque
from xml.dom import minidom
import re

def ch(node, childtype):
    return node.getElementsByTagName(childtype)[0]

if os.path.exists( Abstracts_file ):
    Abstracts = pickle.load( bz2.BZ2File( Abstracts_file, 'rb' ) )
else:
    # `Abstracts` will be incrementally assembled, by performing multiple queries,
    # each returning at most `retrieve_per_query` entries.
    Abstracts = deque()
    retrieve_per_query = 500
    
    print('Fetching Abstracts of results: ')
    for start in range( 0, len(Ids), retrieve_per_query ):
        if (start % 10000 == 0):
            print('')
            print(start, end='')
        else:
            print('.', end='')
        
        # build comma separated string with the ids at indexes [start, start+retrieve_per_query)
        query_ids = ','.join( [ str(id) for id in Ids[ start : start+retrieve_per_query ] ] )
        
        # issue requests to the server, until we get the full amount of data we expect
        while True:
            try:
                #s = Entrez.read( Entrez.efetch(db="pubmed", id=query_ids, retmode="xml" ) )['PubmedArticle']
                s = minidom.parse( Entrez.efetch(db="pubmed", id=query_ids, retmode="xml" ) ).getElementsByTagName("PubmedArticle")
            except http.client.IncompleteRead:
                print('r', end='')
                continue
            break
        
        i = 0
        for p in s:
            abstr = ''
            if (p.getElementsByTagName('MedlineCitation')):
                citNode = ch(p,'MedlineCitation')
                pmid = ch(citNode,'PMID').firstChild.data
                articleNode = ch(citNode,'Article')
                if (articleNode.getElementsByTagName('Abstract')):
                    try:
                        abstr = ch(ch(articleNode,'Abstract'),'AbstractText').firstChild.data
                    except AttributeError:
                        abstr = ch(ch(articleNode,'Abstract'),'AbstractText').toprettyxml("  ")
                        abstr = re.sub(r"\s+", " ", re.sub("<[^>]*>", "", abstr))
            elif (p.getElementsByTagName('BookDocument')):
                bookNode = ch(p,'BookDocument')
                pmid = ch(bookNode,'PMID').firstChild.data
                if (bookNode.getElementsByTagName('Abstract')):
                    try:
                        abstr = ch(ch(bookNode,'Abstract'),'AbstractText').firstChild.data
                    except AttributeError:
                        abstr = ch(ch(bookNode,'Abstract'),'AbstractText').toprettyxml("  ")
                        abstr = re.sub(r"\s+", " ", re.sub("<[^>]*>", "", abstr))
            else:
                raise Exception('Unrecognized record type, for id %d (keys: %s)' % (Ids[start+i], str(p.keys())) )
            
            Abstracts.append( (int(pmid), str(abstr)) )
            i += 1
    
    # Save Abstracts, as a dictionary indexed by Ids
    Abstracts = dict( Abstracts )
    
    pickle.dump( Abstracts, bz2.BZ2File( Abstracts_file, 'wb' ) )

Taking a look at one paper's abstract:

In [26]:
Abstracts[33017218]

'Entomopathogenic fungi naturally infect insect hosts in environment. Fungal invasion and host immune defense are still in the progress of co-evolution. In this study, entomopathogenic fungus '

## ELink: Searching for related items in NCBI Entrez

To understand how to obtain paper citations with Entrez, we will first assemble a small set of PubMed IDs, and then query for their citations.
To that end, we search here for papers published in the Nature journal with our given keyword.

In [27]:
CA_search_term = search_term+'[TIAB] AND Nature[JOUR]'
CA_ids = Entrez.read( Entrez.esearch( db="pubmed", term=CA_search_term ) )['IdList']
CA_ids

['32807960', '32218545', '31597961', '31576029', '30651637', '30455422', '29995855', '29643509', '29236691', '29236684', '28425998', '28024298', '27762375', '27708306', '27121815', '27027296', '27027290', '27027285', '26108828', '25971507']

In [28]:
CA_summ = {
    p['Id'] : ( p['Title'], p['AuthorList'], p['PubDate'][:4], p['FullJournalName'], p.get('DOI', '') )
    for p in Entrez.read( Entrez.esummary(db="pubmed", id=','.join( CA_ids )) )
    }
CA_summ

{'32807960': ('Alarm as devastating banana fungus reaches the Americas.',
  ['Lambert J'],
  '2019',
  'Nature',
  '10.1038/d41586-019-02489-5'),
 '32218545': ('Skin-eating fungus is mighty species slayer.',
  ['Warren M'],
  '2019',
  'Nature',
  '10.1038/d41586-019-01002-2'),
 '31597961': ('A sensor kinase controls turgor-driven plant infection by the rice blast fungus.',
  ['Ryder LS', 'Dagdas YF', 'Kershaw MJ', 'Venkataraman C', 'Madzvamuse A', 'Yan X', 'Cruz-Mireles N', 'Soanes DM', 'Oses-Ruiz M', 'Styles V', 'Sklenar J', 'Menke FLH', 'Talbot NJ'],
  '2019',
  'Nature',
  '10.1038/s41586-019-1637-x'),
 '31576029': ("CRISPR might be the banana's only hope against a deadly fungus.",
  ['Maxmen A'],
  '2019',
  'Nature',
  '10.1038/d41586-019-02770-7'),
 '30651637': ('A kiwellin disarms the metabolic activity of a secreted fungal virulence factor.',
  ['Han X', 'Altegoer F', 'Steinchen W', 'Binnebesel L', 'Schuhmacher J', 'Glatter T', 'Giammarinaro PI', 'Djamei A', 'Rensing SA', 'Rei

Because we restricted our search to papers in an open-access journal, you can then follow their DOIs to freely access their PDFs at the journal's website.

We will now issue calls to `Entrez.elink()` using these PubMed IDs, to retrieve the IDs of papers that cite them.
The database from which the IDs will be retrieved is [PubMed Central](http://www.ncbi.nlm.nih.gov/pmc/), a free digital database of full-text scientific literature in the biomedical and life sciences.

A complete list of the kinds of links you can retrieve with `Entrez.elink()` can be found [here](http://eutils.ncbi.nlm.nih.gov/entrez/query/static/entrezlinks.html).

In [29]:
CA_citing = {
    id : Entrez.read( Entrez.elink(
            cmd = "neighbor",               # ELink command mode: "neighbor", returns
                                            #     a set of UIDs in `db` linked to the input UIDs in `dbfrom`.
            dbfrom = "pubmed",              # Database containing the input UIDs: PubMed
            db = "pmc",                     # Database from which to retrieve UIDs: PubMed Central
            LinkName = "pubmed_pmc_refs",   # Name of the Entrez link to retrieve: "pubmed_pmc_refs", gets
                                            #     "Full-text articles in the PubMed Central Database that cite the current articles"
            from_uid = id                   # input UIDs
            ) )
    for id in CA_ids
    }

In [30]:
CA_citing['27027285']

[{'LinkSetDbHistory': [], 'ERROR': [], 'LinkSetDb': [{'Link': [{'Id': '7471981'}, {'Id': '6693483'}], 'DbTo': 'pmc', 'LinkName': 'pubmed_pmc_refs'}], 'DbFrom': 'pubmed', 'IdList': ['27027285']}]

We have in `CA_citing[paper_id][0]['LinkSetDb'][0]['Link']` the list of papers citing `paper_id`. To get it as just a list of ids, we can do

In [31]:
cits = [ l['Id'] for l in CA_citing['27027285'][0]['LinkSetDb'][0]['Link'] ]
cits

['7471981', '6693483']

However, one more step is needed, as what we have now are PubMed Central IDs, and not PubMed IDs. Their conversion can be achieved through an additional call to `Entrez.elink()`:

In [32]:
cits_pm = Entrez.read( Entrez.elink( dbfrom="pmc", db="pubmed", LinkName="pmc_pubmed", from_uid=",".join(cits)) )
cits_pm

[{'LinkSetDbHistory': [], 'ERROR': [], 'LinkSetDb': [{'Link': [{'Id': '32722029'}, {'Id': '31440220'}], 'DbTo': 'pubmed', 'LinkName': 'pmc_pubmed'}], 'DbFrom': 'pmc', 'IdList': ['7471981', '6693483']}]

In [33]:
ids_map = { pmc_id : link['Id'] for (pmc_id,link) in zip(cits_pm[0]['IdList'], cits_pm[0]['LinkSetDb'][0]['Link']) }
ids_map

{'7471981': '32722029', '6693483': '31440220'}

And to check these papers:

In [34]:
{   p['Id'] : ( p['Title'], p['AuthorList'], p['PubDate'][:4], p['FullJournalName'], p.get('DOI', '') )
    for p in Entrez.read( Entrez.esummary(db="pubmed", id=','.join( ids_map.values() )) )
    }

{'32722029': ('The Dual Function of the Fungal Toxin Candidalysin during <i>Candida albicans</i>-Macrophage Interaction and Virulence.',
  ['König A', 'Hube B', 'Kasper L'],
  '2020',
  'Toxins',
  '10.3390/toxins12080469'),
 '31440220': ('Protein-Protein Interactions in <i>Candida albicans</i>.',
  ['Schoeters F', 'Van Dijck P'],
  '2019',
  'Frontiers in microbiology',
  '10.3389/fmicb.2019.01792')}

### Citations dataset

We have now seen all the steps required to assemble a dataset of citations to each of the papers in our dataset.

In [35]:
Citations_file = 'data/' + search_term + '_Citations.pkl.bz2'
Citations = []

At least one server query will be issued per paper in `Ids`. Because NCBI allows for at most 3 queries per second (see [here](http://biopython.org/DIST/docs/api/Bio.Entrez-pysrc.html#_open)), this dataset will take a long time to assemble. Should you need to interrupt it for some reason, or the connection fail at some point, it is safe to just rerun the cell below until all data is collected.

In [36]:
import http.client

if Citations == [] and os.path.exists( Citations_file ):
    Citations = pickle.load( bz2.BZ2File( Citations_file, 'rb' ) )

if len(Citations) < len(Ids):
    
    i = len(Citations)
    checkpoint = int(len(Ids) / 100) + 1      # save to hard drive at every 1% of Ids fetched
    
    for pm_id in Ids[i:]:               # either starts from index 0, or resumes from where we previously left off
        
        while True:
            try:
                # query for papers archived in PubMed Central that cite the paper with PubMed ID `pm_id`
                c = Entrez.read( Entrez.elink( dbfrom = "pubmed", db="pmc", LinkName = "pubmed_pmc_refs", id=str(pm_id) ) )
                
                c = c[0]['LinkSetDb']
                if len(c) == 0:
                    # no citations found for the current paper
                    c = []
                else:
                    c = [ l['Id'] for l in c[0]['Link'] ]
                    
                    # convert citations from PubMed Central IDs to PubMed IDs
                    p = []
                    retrieve_per_query = 500
                    for start in range( 0, len(c), retrieve_per_query ):
                        query_ids = ','.join( c[start : start+retrieve_per_query] )
                        r = Entrez.read( Entrez.elink( dbfrom="pmc", db="pubmed", LinkName="pmc_pubmed", from_uid=query_ids ) )
                        # select the IDs. If no matching PubMed ID was found, [] is returned instead
                        p.extend( [] if r[0]['LinkSetDb']==[] else [ int(link['Id']) for link in r[0]['LinkSetDb'][0]['Link'] ] )
                    c = p
            
            except http.client.BadStatusLine:
                # Presumably, the server closed the connection before sending a valid response. Retry until we have the data.
                print('r')
                continue
            except HTTPError:
                print('r')
                continue
            break
        
        Citations.append( (pm_id, c) )
        if (i % 10000 == 0):
            print('')
            print(i, end='')
        if (i % 100 == 0):
            print('.', end='')
        i += 1
        
        if i % checkpoint == 0:
            print('\tsaving at checkpoint', i)
            pickle.dump( Citations, bz2.BZ2File( Citations_file, 'wb' ) )
    
    print('\n done.')
    
    # Save Citations, as a dictionary indexed by Ids
    Citations = dict( Citations )
    
    pickle.dump( Citations, bz2.BZ2File( Citations_file, 'wb' ) )

......	saving at checkpoint 52608
.....	saving at checkpoint 53156
......	saving at checkpoint 53704
.....	saving at checkpoint 54252
.....
 done.


To see that we have indeed obtained the data we expected, you can match the ids below, with the ids listed at the end of last section.

In [41]:
Citations[27027285]

[32722029, 31440220]

## Where do we go from here?

Running the code above generates multiple local files, containing the datasets we'll be working with. Loading them into memory is a matter of just issuing a call like<br>
``data = pickle.load( bz2.BZ2File( data_file, 'rb' ) )``.

The Entrez module will therefore no longer be needed, unless you wish to extend your data processing with additional information retrieved from PubMed.

Should you be interested in looking at alternative ways to handle the data, have a look at the [sqlite3](http://docs.python.org/3/library/sqlite3.html) module included in Python's standard library, or [Pandas](http://pandas.pydata.org/), the Python Data Analysis Library.